# Twitter filterer
This notebook allows to filter tweets. One can filter for a certain text that must be contained or that is must not be contained in the message. One can also filter for a maximum amount, for a time frame and one can remove tweets with redundant messages.

In [ ]:
import json
import math
from datetime import datetime, timezone

In [ ]:
def load_tweets(filenames):
    """
    Loads the tweets from the file with the given name into an array of tweets.

    @param filename: The filename of the file to load the tweets from.

    @returns: An array of tweets.
    """
    tweets = []
    for filename in filenames:
        with open(filename, 'r') as f:
            for sLine in f:
                tweet = json.loads(sLine)
                tweets.append(tweet)
    return tweets

def get_hashtags(tweet):
    """
    Extracts the associated hashtags of tweet.

    @param tweet: The tweet, which is in the tweepy json format, and which we wish to extract its associated hashtags.

    @returns: list of hashtags (in lower case)
    """
    entities = tweet.get('entities', {})
    hashtags = entities.get('hashtags', [])

    return [tag['tag'].lower() for tag in hashtags]

In [ ]:
# The filename of the file to store the tweets into
all_twitter_fields_filename = ["../data/coronation_2023_05_05.json", "../data/coronation_2023_05_05_2.json", "../data/coronation_2023_05_05_3.json"]

In [ ]:
tweets = load_tweets(all_twitter_fields_filename)

In [ ]:
# The filename of the file to store the filtered tweets
filtered_tweet_fields_filename = "../data/coronation_2023_05_05_filtered.json"

# The fields of interest
fields_of_interest = ['id', 'text', 'author_id', 'context_annotations', 'conversation_id', 'created_at', 'entities', 'geo', 'in_reply_to_user_id', 'lang', 'referenced_tweets']

# What we want our tweets to contain
# filter_for = "charles"

# What the tweets should NOT contain (as a hashtag or a word in the text)
filter_out = []

# The amount of tweets we want to filter out
amount_of_tweets = math.inf

# Whether tweets with redundant text should be removed or not
remove_redundant_tweets_texts = True

# Date constraints
start_date = datetime.fromisoformat('2023-05-05T00:00:00.000Z'[:-1]).astimezone(timezone.utc)
end_date = datetime.fromisoformat('2023-05-05T23:59:59.000Z'[:-1]).astimezone(timezone.utc)

I load the posts from the file with all fields and delete all fields I am not interested in. Then I store the tweets with the remaining fields if they contain the filter_for and the amount is not exceeded.

In [ ]:
# from collections import Counter

with open(filtered_tweet_fields_filename, 'w') as fOut:
    count = 0
    tweet_texts = []
    # place_ids_counter = Counter()

    for tweet in tweets:

        # Remove not interesting fields
        for key in list(tweet.keys()):
            if key not in fields_of_interest:
                del tweet[key]

        # Remove tweets which do not contain the filter_for text
        text = tweet.get("text").lower()
        # if filter_for not in text:
        #     continue

        # Remove tweets which contain any of filter_out in text or hashtag
        hashtags = get_hashtags(tweet)
        hashtags_string = " ".join(hashtags)
        if any(word in (text + hashtags_string) for word in filter_out):
            continue

        # Remove redundant text tweets
        if remove_redundant_tweets_texts:
            if text not in tweet_texts:
                tweet_texts.append(text)
                count = count + 1
            else:
                continue

        # Filter out date
        created_at = tweet.get('created_at', '')
        date = datetime.fromisoformat(created_at[:-1]).astimezone(timezone.utc)
        date_in_range = start_date < date < end_date
        if not date_in_range:
            continue

        # Take no more than amount_of_tweets
        if count > amount_of_tweets:
            break

        # Store tweet again
        fOut.write("{}\n".format(json.dumps(tweet)))

print("Filtered tweets successfully stored to: ", filtered_tweet_fields_filename)